In [1]:
import os
import logging
import sys
import random
import numpy as np
import random
import inspect
import cv2

from datetime import datetime
from abc import ABC, abstractmethod
from collections import deque

import tensorflow as tf

from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.layers import Dense, Conv2D, Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop

import gym
from gym.wrappers import AtariPreprocessing, FrameStack

from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
#globals
MINIBATCH_SIZE = 32
REPLAY_MEMORY_SIZE = 1000000
AGENT_HISTORY_LENGTH = 4
TARGET_NETWORK_UPDATE_FREQUENCY = 10000
DISCOUNT_FACTOR = 0.99
FRAME_SKIP = 4 #ACTION_REPEAT
UPDATE_FREQUENCY = 4
LEARNING_RATE = 0.00025
GRADIENT_MOMENTUM = 0.95
SQUARED_GRADIENT_MOMENTUM = 0.95
MIN_SQUARED_GRADIENT = 0.01
INITIAL_EXPLORATION = 1.
FINAL_EXPLORATION = 0.1
FINAL_EXPLORATION_FRAME = 1000000
REPLAY_START_SIZE = 50000
NO_OP_MAX = 30

In [5]:
env = gym.make('PongNoFrameskip-v4')

env = AtariPreprocessing(env, noop_max=NO_OP_MAX, frame_skip=FRAME_SKIP, screen_size=84, terminal_on_life_loss=True, grayscale_obs=True,
                 scale_obs=False)

env = FrameStack(env, num_stack=4)

In [6]:
def format_obs(obs):
    return np.moveaxis(np.array(obs), 0, -1)

In [14]:
import matplotlib.pyplot as plt
import time

In [18]:
obs1 = env.reset()
obs2, reward, done, _ = env.step(env.action_space.sample())


In [37]:
obs3, reward, done, _ = env.step(env.action_space.sample())


In [38]:
obs4, reward, done, _ = env.step(env.action_space.sample())


In [39]:
for i in range(3):
    print(np.all(obs3[i,:,:] ==  obs3[i+1,:,:]))

False
False
False


In [40]:
for i in range(4):
    for j in range(4):
        print(np.all(obs3[j,:,:] ==  obs4[i,:,:]))
    print("---------------")
    


False
True
False
False
---------------
False
False
True
False
---------------
False
False
False
True
---------------
False
False
False
False
---------------


In [41]:
np.all(obs3[1:4,:,:] == obs4[0:3,:,:])

True

In [15]:
class DQN(object):
    def __init__(self, input_shape, output_units, save_model_dir='models', save_model_name='model.h5',
                 load_model_dir=None, load_model_name=None):
        self.input_shape = input_shape
        self.output_units = output_units
        self.save_model_dir = save_model_dir
        self.save_model_name = save_model_name
        self.load_model_dir = load_model_dir
        self.load_model_name = load_model_name
        self.model = self._load_model()

        if not os.path.exists(self.save_model_dir):
            os.makedirs(self.save_model_dir)

    def _load_model(self):
        if self.load_model_dir is None or self.load_model_name is None:
            print("Creating new neural-network")
            return self.get_q_network()

        model_name = os.path.join(self.load_model_dir, self.load_model_name)

        if os.path.exists(model_name):
            print("Loading existing model, " + str(model_name))
            return load_model(model_name)

        raise Exception("Model could not be loaded.")

    @abstractmethod
    def get_q_network(self):
        pass

    def get_prediction(self, preprocessed_input):
        return self.model.predict(np.expand_dims(preprocessed_input, 0))[0]

    def get_predicted_action(self, preprocessed_input):
        return np.argmax(self.get_prediction(preprocessed_input))

    def prepare_minibatch(self, transitions_minibatch, gamma):
        expected_output_minibatch = []
        input_minibatch = []

        for current_input, action, reward, next_input, is_terminal_state in transitions_minibatch:
            q_value = reward
            if not is_terminal_state:
                q_value += gamma * np.amax(self.get_prediction(next_input))
            prediction = self.get_prediction(current_input)
            prediction[action] = q_value
            expected_output_minibatch.append(prediction)
            input_minibatch.append(current_input)

        expected_output_minibatch = np.array(expected_output_minibatch)
        input_minibatch = np.array(input_minibatch)

        return input_minibatch, expected_output_minibatch

    def perform_gradient_descent_step(self, _input, _output):
        self.model.fit(x=_input, y=_output, epochs=1, verbose=0)

    def save_model(self, step=''):
        model_name = os.path.join(self.save_model_dir, (str(step) + '--' + self.save_model_name))
        self.model.save(model_name)

In [35]:
EXPERIENCE_REPLAY_MEMORY = deque([], maxlen=1000000)
EPSILON_DECAY_RATE = 9.000000000000001e-07

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir=logdir)

In [ ]:
def update_epsilon(self):
    if self.epsilon < 0.1:
        self.epsilon = 0.1
        return
    elif self.epsilon == 0.1:
        return
    else:
        self.epsilon -= self.epsilon_decay_rate

In [ ]:
def e_greedy_select_action(self, preprocessed_input):
    if random.random() <= self.epsilon:
        action = self.env_manager.get_random_action()
    else:
        action = self.DQN.get_predicted_action(preprocessed_input)

    update_epsilon()

    return action

In [49]:
def initialize_replay_memory():
    frame_count = 0
    done = True
    
    while frame_count < REPLAY_START_SIZE:
        if done:
            current_obs = env.reset()
            done = False
            
        action = env.action_space.sample()
        next_obs, reward, done, info = env.step(action)
        current_obs = format_obs(current_obs)
        next_obs = format_obs(next_obs)
        EXPERIENCE_REPLAY_MEMORY.append((current_obs, action, reward, next_obs, done))
        
        frame_count += 1
        clear_output(wait=True)
        print("{:.1f}% done.".format(frame_count/float(REPLAY_START_SIZE) * 100 ))
        
    clear_output(wait=True)
    print("Replay memory initialized.")


In [61]:
def 

array([1, 1])

In [62]:
m = deque(a)

In [34]:
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
while True:
    obs = env.reset()
    done = False
    while not done:
        env.render()
        obs = format_obs(obs)
        cv2.imshow('frame', obs[:, :, 0])
        cv2.waitKey(100) & 0xFF
        action = env.action_space.sample()
        # print(action)
        obs, reward, done, info = env.step(action)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [5]:
i = -100

In [6]:
7 if i > 0 else 0 if i == 0 else -1

-1

In [11]:
env.reset?

Signature: env.reset(**kwargs)
Docstring:
Resets the state of the environment and returns an initial observation.

Returns:
    observation (object): the initial observation.
File:      ~/anaconda3/envs/rl-gpu/lib/python3.7/site-packages/gym/wrappers/frame_stack.py
Type:      method
